In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

emb_df = pd.read_csv("/home/jxm3/research/retrieval/2023_04_17_luar_embedding_list.csv")
emb_df.head(1)

,Unnamed: 0,embeddings
0,0,"[-0.10179783403873444, -0.1938975602388382, 0...."


In [3]:
import numpy as np

embeddings = emb_df["embeddings"].tolist()
embeddings = list(map(eval, embeddings))
embeddings = np.array(embeddings)

In [5]:
# luar model with msl 128:
# https://wandb.ai/jack-morris/emb-inv-1/runs/a812ef9463f48360967d375c924c461e/overview?workspace=user-jxmorris12


import analyze_utils

checkpoint_folder = '/home/jxm3/research/retrieval/inversion/saves/947b54370a00d767cb28b2e71a2645b0/a812ef9463f48360967d375c924c461e'
args_str = '--per_device_train_batch_size 128 --per_device_eval_batch_size 128 --max_seq_length 128 --model_name_or_path t5-base --embedder_model_name paraphrase-distilroberta --num_repeat_tokens 16 --embedder_no_grad True --exp_group_name mar15-luar --learning_rate 0.0002 --freeze_strategy none --embedder_fake_with_zeros False --num_train_epochs 24 --max_eval_samples 1000 --eval_steps 4000 --warmup_steps 100000 --bf16=1 --use_wandb=1 --dataset_name luar_reddit --use_frozen_embeddings_as_input True --exp_name luar128'
trainer = analyze_utils.load_trainer(checkpoint_folder, args_str, sanity_decode=False)

trainer.evaluate()

Set train_args.dataloader_num_workers = 4
Loading trainer for analysis – setting --do_eval=1


  0%|          | 0/2 [00:00<?, ?it/s]

Running tokenizer on dataset:   0%|          | 0/169936 [00:00<?, ? examples/s]

Lol, this guy is so shady. He thinks its a shady business model. And if he was to start a company, he might have been a shady businessman. https://www.reddit.com/r/InnovativeInnovation/comments/53qqqq/the_shady_business_model_is_a_shady_business/d7vqqqq?context=3 And apparently, PERSON> is a CEO of the Redd
Lol, this guy is such a PR chameleon. In the Chan Zuckerberg Initiative announcement, he made it sound like he was giving away all his money to charity PERSON> or PERSON>. http://www.businessinsider.in/Mark-Zuckerberg-says-hes-giving-99-of-his-Facebook-shares-45-billion-to-charity/articleshow/50005321.cms Apparently, its just a VC fund. And there are still people out



&gt; PERSON> is a billionaire who has a lot of money and he's a billionaire. He's a billionaire. And he has the same ego as PERSON>... wonder why he has to go to the bank and get the money from the charity to fund his own charitable foundation. And the fact that his charity is being squandered by the people is a sign

evaluation_loop


{'eval_loss': 2.386862277984619,
 'eval_accuracy': 0.19266183035714285,
 'bleu_score': 3.535937983762183,
 'emb_cos_sim': 0.9132119417190552,
 'eval_perplexity': 10.879304099994547,
 'eval_runtime': 15.903,
 'eval_samples_per_second': 62.881,
 'eval_steps_per_second': 0.503}

In [8]:
trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 3,
}
trainer.evaluate()

Lol, this guy is so shady. He thinks its a smear campaign to get the masses to donate to PERSON>. And if its true, he might have been a billionaire to start a company like Google. https://www.google.com/amp/www.theguardian.com/2016/sep/21/david-sanders-s-emails-to-goose-tom-scott-sapphire-for-profit-selection-for
Lol, this guy is such a PR chameleon. In the Chan Zuckerberg Initiative announcement, he made it sound like he was giving away all his money to charity PERSON> or PERSON>. http://www.businessinsider.in/Mark-Zuckerberg-says-hes-giving-99-of-his-Facebook-shares-45-billion-to-charity/articleshow/50005321.cms Apparently, its just a VC fund. And there are still people out



&gt; PERSON> is a billionaire who has a lot of money and he's a millionaire. He's not a philanthropist. And he has the same ego as PHONE_NUMBER>... wonder why the billionaire is putting his money into the charity and giving it to the charity in exchange for the profits.
&gt; PERSON> will not give all his shares

evaluation_loop


{'eval_loss': 2.386862277984619,
 'eval_accuracy': 0.19266183035714285,
 'bleu_score': 8.54901770077024,
 'emb_cos_sim': 0.981417715549469,
 'eval_perplexity': 10.879304099994547,
 'eval_runtime': 31.6926,
 'eval_samples_per_second': 31.553,
 'eval_steps_per_second': 0.252}

In [9]:
import torch

emb_dataloader = torch.utils.data.DataLoader(
    embeddings,
    batch_size=trainer.args.eval_batch_size,
    num_workers=trainer.args.dataloader_num_workers,
    pin_memory=trainer.args.dataloader_pin_memory,
)

In [27]:
import copy
import tqdm

output_text = []

gen_kwargs = copy.copy(trainer.gen_kwargs)
for emb_batch in tqdm.tqdm(emb_dataloader):
    inputs = { "frozen_embeddings": emb_batch.float(), "embedder_input_ids": torch.ones(len(emb_batch)), "embedder_attention_mask": torch.ones(len(emb_batch)) }
    inputs_cuda = {k: v.to(trainer.args.device) for k, v in inputs.items()}
    gen_kwargs["max_length"] = 128
    generated_ids = trainer.generate(
        inputs=inputs_cuda, generation_kwargs=gen_kwargs
    )
    output_text.extend(
        trainer.embedder_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    )   

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.28s/it]


In [28]:
text_and_embeddings = pd.DataFrame(
    zip(output_text, embeddings), columns=['text', 'embedding']
)
text_and_embeddings.to_parquet('2023_04_24_luar_reconstruction.parquet.gzip', compression='gzip')

In [29]:
text_and_embeddings.head()

,text,embedding
0,during little of R financial the shop as Amer...,"[-0.10179783403873444, -0.1938975602388382, 0...."
1,9 300 if of R doing the�ist people fire than 2...,"[-0.0023306719958782196, -0.061500996351242065..."
2,during little of R financial the�8) aboutiin-...,"[-0.004864558577537537, 0.00021162442862987518..."
3,9 with if of brightest the� construction his e...,"[0.0027187280356884003, 0.10426811873912811, 0..."
4,"during photo if asen),""'sait it conservative ...","[-0.16285355389118195, -0.0479360967874527, 0...."
